In [ ]:
! pip install datasets sentencepiece transformers evaluate rouge_score bert_score huggingface_hub sentencepiece
! pip3 install - U scikit-learn scipy matplotlib


In [1]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu
import torch
import torch.nn as nn
from transformers import pipeline


In [2]:
from datasets import load_dataset

data_files = {"train": "./datasets/train.pkl",
              "test": "./datasets/test.pkl",
              "val": "./datasets/val.pkl"}

raw_dataset = load_dataset("pandas", data_files=data_files, cache_dir=None)

train_data = raw_dataset['train']
test_data = raw_dataset['test']
val_data = raw_dataset['val']


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Dataset pandas downloaded and prepared to /Users/madhavappaneni/.cache/huggingface/datasets/pandas/default-7c097623fefc7168/0.0.0/3ac4ffc4563c796122ef66899b9485a3f1a977553e2d2a8a318c72b8cc6f2202. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(51692, 1)
(6435, 1)
(6589, 1)


In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "t5-small"

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


In [5]:
def tokenize_function(examples):
    inputs = [ex["input_text"] for ex in examples['conversation']]
    targets = [ex["label"] for ex in examples['conversation']]
    model_inputs = tokenizer(
        inputs, text_target=targets, truncation=True, padding='max_length'
    )
    return model_inputs

In [6]:
tokenized_datasets = raw_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_dataset["train"].column_names,
)


Map:   0%|          | 0/51692 [00:00<?, ? examples/s]

/Users/madhavappaneni/Library/Python/3.9/lib/python/site-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/6435 [00:00<?, ? examples/s]

Map:   0%|          | 0/6589 [00:00<?, ? examples/s]

In [10]:
# from evaluate import load
# from transformers import pipeline


# # Define the Rouge and BLEU metrics using the load_metric function
# rouge_metric = load('rouge')
# bleu_metric = load('bleu')

# # Define the BertScore metric
# # bertscore_metric = BertScore(metrics=['precision', 'recall', 'f1'], lang="en")
# bertscore_metric = load("bertscore")

# # Define the BLEURT metric
# # bleurt = load("bleurt", module_type="metric")


# def compute_metrics(pred):
#     # Decode the predictions and labels
#     preds = tokenizer.batch_decode(pred.predictions, skip_special_tokens=True)
#     labels = tokenizer.batch_decode(pred.label_ids, skip_special_tokens=True)

#     # Compute the Rouge metric
#     rouge = rouge_metric.compute(predictions=preds, references=labels)

#     # Compute the BLEU metric
#     bleu = bleu_metric.compute(predictions=preds, references=[labels])

#     # Compute the BertScore metric
#     bertscore = bertscore_metric.score(preds, labels, batch_size=2)
#     bertscore = {'bertscore_'+k: v.mean().item() for k, v in bertscore.items()}

#     perplexity = torch.exp(pred.loss)

#     # Compute the BLEURT score
#     # bleurt_score = bleurt(preds, labels)[0]['score']

#     # Combine the metrics
#     metrics = {'rouge': rouge['rouge2'].fmeasure, 'bleu': bleu['score'],
#                'bertscore_precision': bertscore['bertscore_precision'], 'bertscore_recall': bertscore['bertscore_recall'],
#                'bertscore_f1': bertscore['bertscore_f1'],
#                'perplexity': perplexity.item()
#                #    'bleurt_score': bleurt_score
#                }

#     return metrics


In [7]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",  # Save the model at the end of each epoch
    save_total_limit=1,  # Limit the number of saved checkpoints to 3
)


In [8]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
#     compute_metrics=compute_metrics
)


In [13]:
trainer.train()

/projects/academic/courses/cse546s23/skunusot/anaconda3/envs/cuda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,16.542200
20,15.676900
30,14.326100
40,13.350900
50,11.599900
60,9.813700
70,7.187500
80,4.817800
90,2.934800
100,1.628700


TrainOutput(global_step=4580, training_loss=0.31684812062692436, metrics={'train_runtime': 2892.8341, 'train_samples_per_second': 50.648, 'train_steps_per_second': 1.583, 'total_flos': 1.982973938447155e+16, 'train_loss': 0.31684812062692436, 'epoch': 4.0})

In [14]:
# save the model
trainer._save("./t5-finetuned/model")
tokenizer.save_pretrained("./t5-finetuned/tokenizer")


('./t5-finetuned/tokenizer/tokenizer_config.json',
 './t5-finetuned/tokenizer/special_tokens_map.json',
 './t5-finetuned/tokenizer/spiece.model',
 './t5-finetuned/tokenizer/added_tokens.json')

In [15]:
results = trainer.evaluate()

In [16]:
results

{'eval_loss': 0.0964617058634758,
 'eval_runtime': 34.4698,
 'eval_samples_per_second': 165.71,
 'eval_steps_per_second': 5.193,
 'epoch': 4.0}

In [17]:
from huggingface_hub import notebook_login
notebook_login()



In [21]:
model.push_to_hub("t5-small-chit-chat")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/madhavappaneni/t5-small-empathetic-dialogues/commit/c1bf0319e600bb011302f462c17185ce3abca874', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='c1bf0319e600bb011302f462c17185ce3abca874', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
tokenizer.push_to_hub("t5-small-chit-chat")

CommitInfo(commit_url='https://huggingface.co/madhavappaneni/t5-small-empathetic-dialogues/commit/e6ecfc475c4db36c9ce975378ae51b74475db168', commit_message='Upload tokenizer', commit_description='', oid='e6ecfc475c4db36c9ce975378ae51b74475db168', pr_url=None, pr_revision=None, pr_num=None)